## 4.6 제대로 비교하기 위해 유니코드 정규화 하기
- 유니코드에는 결합 문자 존재하기 때문에 문자열 비교 간단하지 않음
- 앞 문자에 연결되는 발음 구별 기호는 인쇄할 때 앞 문자와 하나로 결합되어 출력

In [1]:
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [2]:
len(s1), len(s2)

(4, 5)

In [3]:
s1 == s2

False

- U+0301은 Combining Acute Accent
    - e 다음에 이 문자가 오면 é 만듦
    - 유니코드에서는 "é" 와 "e\u0301"이 동일하다고 하며 이 두 시퀸스를 동일하게 처리해야 함
    - 하지만 **파이썬에서는** 서로 다른 두 개의 코드 포인트 시퀸스롤 보도 동일하지 않다고 판단
    
- unicodedata.normalize() 함수가 제공하는 유니코드 정규화를 이용해야 함


unicodedata.normalize()의 첫번째 인수
- NFC
    - 코드 포인티를 조합해서 가장 짧은 동일한 문자열 생성
- NFD
    - 조합된 문자를 기본 문잗와 별로의 결합 문자로 분리
    

In [4]:
from unicodedata import  normalize

s1 = 'café'
s2 = 'cafe\u0301'


In [5]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [7]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [8]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [9]:
normalize('NFD', s1) == normalize('NFD', s2)

True

- 키보드는 일반적으로 결합된 문자를 입력할 수 있으므로, 사용자가 입력하는 텍스트는 기본적으로 NFC 형태
- 안전을 보장하기 위해 파일에 저장하기 전에 **normalize('NFC', user_text)** 코드로 문자열 청소하는 것이 좋음


- NFC에 의해 다른 문자 하나로 정규화되는 문자 있음
    - 전기저항을 나타내는 옴 기호는 그리스어 대문자 오메가로 정규화됨
    - 겉모습은 똑같지만 다르다고 판단되므로 정규화해서 뜻하지 않은 문제 예방

In [10]:
from unicodedata import normalize, name
ohm = "\u2126"
name(ohm)

'OHM SIGN'

In [13]:
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

- NFKC, NFKD에서 k는 호환성(compatibility)를 의미
    - 정규화의 더 강력한 형태

In [16]:
from unicodedata import normalize, name
half = "½" # ㅊ+한자

### 4.6.1 케이스 폴딩
- 기본적으로 모든 텍스트를 소문자로 변환하는 연산, 얀갼의 벼놘을 동반
- python s.s에 추가되 str.casefold() 매서드 이용

<br>

- latin1 문자만 담고 있는 문자열 s의 경우 s.casecade와 s.lower()을 실행한 결과과 동일 
- 아래의 예시는 다름

<hr>

In [26]:
"\u00B5"

'µ'

In [30]:
micro = 'µ' # "\u00B5"
name(micro)

'MICRO SIGN'

In [31]:
micro_cf = micro.casefold()
name(micro_cf)

'GREEK SMALL LETTER MU'

In [32]:
micro, micro_cf

('µ', 'μ')

In [34]:
"\u00DF"

'ß'

In [33]:
eszett = 'ß' # u"\u00DF"
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [35]:
eszett_cv = eszett.casefold()
eszett, eszett_cv

('ß', 'ss')

- python 3.4에서는 str.casefold와 str.lower()가 서로 다른 문자를 반환하는 코드 포인트 116개 존재

### 4.6.2 정규화된 텍스트 매칭을 위한 유틸리티 함수
- NFC, NFD 안전하며 유니코드 문자열을 적절히 비교할 수 있게 해줌
- NFC는 대부분 애플리케이션에서 사용할 수 있는 최고의 정규화된 형태
- str.casefold() 는 대소문자 구분 없이 문자를 비교할 때 가장 좋음

#### 예제 4-13

In [41]:
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize("NFC", str1) == normalize("NFC",str2)

def fold_equal(str1, str2):
    return (normalize("NFC", str1).casefold() == normalize("NFC", str2).casefold())

In [42]:
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2

False

In [43]:
nfc_equal(s1, s2)

True

In [45]:
nfc_equal("A", "a")

False

In [47]:
s3 = "Straße"
s4 = "Strasse"

s3 == s4

False

In [48]:
nfc_equal(s3, s4)

False

In [49]:
fold_equal(s3, s4)

True

In [52]:
fold_equal(s1, s2)

True

In [51]:
fold_equal("A", "a")

True

### 4.6.3 극단적인 "정규화" : 바음 구별 기호 제거

#### 예제 4-14

In [72]:
import unicodedata
import string

def shave_marks(txt):
    
    norm_txt = unicodedata.normalize("NFD", txt)
    print("norm_txt : ", norm_txt)
    shaved = "".join(c for c in norm_txt
                        if not unicodedata.combining(c))
    print([c for c in norm_txt ])
    print([c for c in norm_txt if not unicodedata.combining(c)])
    
    return unicodedata.normalize("NFC", shaved)

In [73]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

norm_txt :  “Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”
['“', 'H', 'e', 'r', 'r', ' ', 'V', 'o', 'ß', ':', ' ', '•', ' ', '½', ' ', 'c', 'u', 'p', ' ', 'o', 'f', ' ', 'Œ', 't', 'k', 'e', 'r', '™', ' ', 'c', 'a', 'f', 'f', 'e', '̀', ' ', 'l', 'a', 't', 't', 'e', ' ', '•', ' ', 'b', 'o', 'w', 'l', ' ', 'o', 'f', ' ', 'a', 'c', '̧', 'a', 'i', '́', '.', '”']
['“', 'H', 'e', 'r', 'r', ' ', 'V', 'o', 'ß', ':', ' ', '•', ' ', '½', ' ', 'c', 'u', 'p', ' ', 'o', 'f', ' ', 'Œ', 't', 'k', 'e', 'r', '™', ' ', 'c', 'a', 'f', 'f', 'e', ' ', 'l', 'a', 't', 't', 'e', ' ', '•', ' ', 'b', 'o', 'w', 'l', ' ', 'o', 'f', ' ', 'a', 'c', 'a', 'i', '.', '”']


'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [74]:
Greek = 'Ζέφυρος, Zéfiro'
shave_marks(Greek)

norm_txt :  Ζέφυρος, Zéfiro
['Ζ', 'ε', '́', 'φ', 'υ', 'ρ', 'ο', 'ς', ',', ' ', 'Z', 'e', '́', 'f', 'i', 'r', 'o']
['Ζ', 'ε', 'φ', 'υ', 'ρ', 'ο', 'ς', ',', ' ', 'Z', 'e', 'f', 'i', 'r', 'o']


'Ζεφυρος, Zefiro'

#### 예제 4-16

In [76]:
def shave_marks_latin(txt):
    norm_txt = unicodedata.normalize('NFD', txt) # 모든 문자를 기반 문자와 결합 표시 기호로 분리
    latin_base = False
    preserve = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # 기반 문자가 라틴 문자일 때 결합 표시기호 건너뛰기
            continue  
        preserve.append(c)    # 아니면 보관
        
        if not unicodedata.combining(c):              # 새로운 기반 문자 찾아내고 라틴 문자인지 판단
            latin_base = c in string.ascii_letters
    shaved = ''.join(preserve)
    
    return unicodedata.normalize('NFC', shaved)   # 문자들을 결합하고 NFC 방식으로 정규화

In [77]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

norm_txt :  “Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”
['“', 'H', 'e', 'r', 'r', ' ', 'V', 'o', 'ß', ':', ' ', '•', ' ', '½', ' ', 'c', 'u', 'p', ' ', 'o', 'f', ' ', 'Œ', 't', 'k', 'e', 'r', '™', ' ', 'c', 'a', 'f', 'f', 'e', '̀', ' ', 'l', 'a', 't', 't', 'e', ' ', '•', ' ', 'b', 'o', 'w', 'l', ' ', 'o', 'f', ' ', 'a', 'c', '̧', 'a', 'i', '́', '.', '”']
['“', 'H', 'e', 'r', 'r', ' ', 'V', 'o', 'ß', ':', ' ', '•', ' ', '½', ' ', 'c', 'u', 'p', ' ', 'o', 'f', ' ', 'Œ', 't', 'k', 'e', 'r', '™', ' ', 'c', 'a', 'f', 'f', 'e', ' ', 'l', 'a', 't', 't', 'e', ' ', '•', ' ', 'b', 'o', 'w', 'l', ' ', 'o', 'f', ' ', 'a', 'c', 'a', 'i', '.', '”']


'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [78]:
shave_marks_latin(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [79]:
Greek = 'Ζέφυρος, Zéfiro'
shave_marks(Greek)

norm_txt :  Ζέφυρος, Zéfiro
['Ζ', 'ε', '́', 'φ', 'υ', 'ρ', 'ο', 'ς', ',', ' ', 'Z', 'e', '́', 'f', 'i', 'r', 'o']
['Ζ', 'ε', 'φ', 'υ', 'ρ', 'ο', 'ς', ',', ' ', 'Z', 'e', 'f', 'i', 'r', 'o']


'Ζεφυρος, Zefiro'

In [80]:
shave_marks_latin(Greek)

'Ζέφυρος, Zefiro'

#### 예제 4-17 
- 서양 활자(타이포그래픽) 기호를 아스키로 변환하기

In [83]:
single_map = str.maketrans("""‚ƒ„ˆ‹‘’“”•–—˜›""",  # <1>
                           """'f"^<''""---~>""")

multi_map = str.maketrans({  # <2>
    '€': 'EUR',
    '…': '...',
    'Æ': 'AE',
    'æ': 'ae',
    'Œ': 'OE',
    'œ': 'oe',
    '™': '(TM)',
    '‰': '<per mille>',
    '†': '**',
    '‡': '***',
})

multi_map.update(single_map)  # <3>

def dewinize(txt):
    return txt.translate(multi_map)

def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))
    no_marks = no_marks.replace("ß", "ss")
    return unicodedata.normalize("NFKC", no_marks)

#### 예제 4-18

In [87]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [88]:
asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

### 4.7 유니코드 텍스트 정렬
- python 은 문자열을 정렬 할 때 각 단어의 코드 포인트를 비교
    - 비아스키 문자를 사용하는 경우 부적절한 결과 발생

In [89]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

- 파이썬에서는 비아스키 텍스트는 locale.strxfrm() 함수를 이용해서 변환

#### 예제 4-19

In [91]:
import locale

locale.setlocale(locale.LC_COLLATE, "pt_BR.UTF-8") # setlocale(LC_COLLATE, <지역언어>)

'pt_BR.UTF-8'

In [92]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits, key=locale.strxfrm)

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

### 유니코드 대조 알고리즘을 이용한 정렬

In [101]:
import pyuca
coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

## 4.8 유니코드 데이터 베이스
- 유니코드 표준은 수많은 구조화된 텍스트 파일의 형태로 하나의 완전한 데이터베이스를 제공
    - 이 데이터베이스에는 코드 포인트를 문자명으로 매핑하는 테이블 뿐만이 아니라 각 문자에 대한 메티데이터 및 각 문자의 연관 방법을 담음
    

#### 예제 4-21
- 유니코드 데이터베이스 수치형 문자 메타 데이터 사용 예

In [105]:
import unicodedata
import re

re_digit = re.compile(r"\d")
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print(f'U+{ord(char):04x}',                       
          char.center(6),                             # 길이가 6인 str의 중앙에 놓인 문자
          're_dig' if re_digit.match(char) else '-',  # r"\d" 정규표현식과 일치하는 문자의 걍우 re_dig 표시
          'isdig' if char.isdigit() else '-',         # char.isdigit()가 참이면 isdig 표시
          'isnum' if char.isnumeric() else '-',       # char.isnumeric() 참이면 isnum 표시
          f'{unicodedata.numeric(char):5.2f}',        # 전체 너비는 5칸이며 소수점 2자리까지 포맷한 숫자값
          unicodedata.name(char),                     # 유니코드 문자명
          sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


## 4.9 이중 모드 str 및 bytes API

### 4.9.1 정규표현식에서의 str과 bytes
- bytes로 정규표현식을 만들면 \d, \w 같은 패턴은 아스키 문자만 매칭
- str로 이 패턴을 만들면 아스키 문자외에 유니코드 숫자나 문자도 매칭


#### 예제 4.22

In [106]:
import re

re_numbers_str = re.compile(r'\d+')     # str 형
re_words_str = re.compile(r'\w+')       # str 형
re_numbers_bytes = re.compile(rb'\d+')  # bytes 형
re_words_bytes = re.compile(rb'\w+')    # bytes 형

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"  # <3>
            " as 1729 = 1³ + 12³ = 9³ + 10³.")        # <4>

text_bytes = text_str.encode('utf_8')  # <5>

print(f'Text\n  {text_str!r}')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))      # <6>
print('  bytes:', re_numbers_bytes.findall(text_bytes))  # <7>
print('Words')
print('  str  :', re_words_str.findall(text_str))        # <8>
print('  bytes:', re_words_bytes.findall(text_bytes))    # <9>

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


## 4.9.2 os 모듈 함수에서 str과 bytes

#### 예제 4-23

In [108]:
import os
os.listdir(".")

['.ipynb_checkpoints',
 '1_1.py',
 '1_2.py',
 '2_1.py',
 '2_3.py',
 '2_4.py',
 '2_5.py',
 '2_6.py',
 '2_py',
 '4.6~.ipynb']

In [109]:
os.listdir(b".")

[b'.ipynb_checkpoints',
 b'1_1.py',
 b'1_2.py',
 b'2_1.py',
 b'2_3.py',
 b'2_4.py',
 b'2_5.py',
 b'2_6.py',
 b'2_py',
 b'4.6~.ipynb']